# Installing Libraries

In [ ]:
!pip install 'accelerate>=0.26.0'
!pip install chromadb
!pip install langchain_chroma langchain_community
!pip install unstructured
!pip install --upgrade --quiet PyMuPDF langchain_community langchain_chroma
!pip install sentence_transformers

# Importing Libraries

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from glob import glob
import os
from tqdm import tqdm

import chromadb
from chromadb.config import Settings
from langchain_chroma import Chroma
#from langchain_unstructured import UnstructuredLoader

from langchain_community.document_loaders import PyMuPDFLoader,PyPDFLoader,PyPDFDirectoryLoader
from langchain.document_loaders import UnstructuredExcelLoader
from langchain.document_loaders import TextLoader
from langchain.document_loaders import UnstructuredWordDocumentLoader

# from langchain_community.document_loaders import ExcelLoader
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)
from langchain_text_splitters import RecursiveCharacterTextSplitter

import chromadb
from chromadb.config import Settings
from openai import AzureOpenAI

# Categorizing types of files

In [3]:
path = "/home/aravichandran_umassd_edu/fuchs_hackathon/hackathon-fall24/startup_packet/synthetic_files/reports_2024-09-26_00-39-17"
txt_files = glob(path+"/*.txt")
xlsx_files = glob(path+"/*.xlsx")
docx_files = glob(path+"/*.docx")
pdf_files = glob(path+"/*.pdf")

files = glob(path+"/*")
file_ext = []
for temp in files:
    ext = temp.split(".")[-1]
    file_ext.append(ext)
np.unique(file_ext)

os.system(f"mkdir -p raw_data/pdf/")
os.system(f"mkdir -p raw_data/txt/")
os.system(f"mkdir -p raw_data/xlsx/")
os.system(f"mkdir -p raw_data/docx/")

for file in tqdm(pdf_files, desc="moving pdfs"):
    os.system(f"cp {file} raw_data/pdf/")

for file in tqdm(txt_files, desc="moving txt"):
    os.system(f"cp {file} raw_data/txt/")

for file in tqdm(docx_files, desc="moving docx"):
    os.system(f"cp {file} raw_data/docx/")

for file in tqdm(xlsx_files, desc="moving xls"):
    os.system(f"cp {file} raw_data/xlsx/")

0

# Parsing Data into Training Vectors

In [2]:
db_path = "parsed_data/db"
os.system(f"mkdir -p {db_path}")
chroma_client = chromadb.PersistentClient(path=db_path)
#chroma_client.delete_collection(name="test1_c")
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

chunked_documents = []

/tmp/ipykernel_1101502/415756860.py:5: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
/home/aravichandran_umassd_edu/miniconda3/envs/fuchs/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
/home/aravichandran_umassd_edu/miniconda3/envs/fuchs/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces`

In [5]:
# Parsing PDF Folder
folder_path = "raw_data/pdf/"
loader = PyPDFDirectoryLoader(folder_path)
document = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunked_documents_temp = text_splitter.split_documents(document)

chunked_documents.extend(chunked_documents_temp)


# Parsing XLSX Folder
folder_path = "raw_data/xlsx/"
xlsx_files = glob(folder_path+"/*")

for file in tqdm(xlsx_files, desc="XLSX Files"):
    loader = UnstructuredExcelLoader(file)
    document = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    chunked_documents_temp = text_splitter.split_documents(document)
    chunked_documents.extend(chunked_documents_temp)

# Parsing TXT Folder
folder_path = "raw_data/txt/"
txt_files = glob(folder_path+"/*")

for file in tqdm(txt_files, desc="TXT Files"):
    loader = TextLoader(file)
    document = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    chunked_documents_temp = text_splitter.split_documents(document)
    chunked_documents.extend(chunked_documents_temp)

    
# Parsing DOCX Folder
folder_path = "raw_data/docx/"
docx_files = glob(folder_path+"/*")

for file in tqdm(docx_files, desc="DOCX Files"):
    loader = UnstructuredWordDocumentLoader(file)
    document = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    chunked_documents_temp = text_splitter.split_documents(document)
    chunked_documents.extend(chunked_documents_temp)

DOCX Files: 100%|██████████| 753/753 [01:26<00:00,  8.71it/s]


## Collating Data

In [7]:
Chroma.from_documents(
    documents=chunked_documents,
    embedding=embedding_function,
    collection_name='test1_c',
    client=chroma_client,
)
print(f"Added {len(chunked_documents)} chunks to chroma db")

Added 12396 chunks to chroma db


# Training Model

ModuleNotFoundError: No module named 'openai'